# GMF and MLP - Rating Data

user one-hot vector와 item one-hot vector는 R에 존재하는 데이터만 활용

input = R에서 존재하는 데이터만 활용 (왜? R에서 존재하는 데이터만 활용하면 결국 user one-hot vector * Latent user vector, item one-hot vector * Latent item vector 을 한것과 동일하게 됨)

GMF Latent user vector = (user_num ,factor)

GMF Latent item vector = (item_num ,factor)

MLP Latent user vector = (user_num ,factor)

MLP Latent item vector = (item_num ,factor)

output_GMF = GMF Latent user vector * GMF Latent item vector

MLP_concat_two_latent_vactors = concat(MLP Latent user vector, MLP Latent item vector)

output_MLP = MLP_layer(MLP_concat_two_latent_vactors)

output = concat(output_GMF, output_MLP)

out = FC_layer(output, 1)

output = Y_ui ~ out

최종 예측 값은 유저가 해당 영화에 줄 평점이 될 것 따라서 이를 바탕으로 영화 추천이 가능

In [ ]:
import os
import time
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.backends.cudnn as cudnn

import pandas as pd
import scipy.sparse as sp
import warnings

warnings.filterwarnings(action='ignore')
%matplotlib inline

In [ ]:
movie_paths = '/content/drive/MyDrive/추천/data/movies/'

movie = pd.read_csv(movie_paths + "ratings.csv")
meta = pd.read_csv(movie_paths + 'movies_metadata.csv', low_memory=False)
meta = meta.rename(columns={'id':'movieId'})

In [ ]:
movie.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
meta.head()

,adult,belongs_to_collection,budget,genres,homepage,movieId,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
movie['movieId'] = movie['movieId'].astype(str)
meta['movieId'] = meta['movieId'].astype(str)

movie = pd.merge(movie, meta[['movieId', 'original_title']], on='movieId')
movie.head()

,userId,movieId,rating,timestamp,original_title
0,1,1371,2.5,1260759135,Rocky III
1,4,1371,4.0,949810302,Rocky III
2,7,1371,3.0,851869160,Rocky III
3,19,1371,4.0,855193404,Rocky III
4,21,1371,3.0,853852263,Rocky III


In [ ]:
df = movie.pivot_table(index='userId', columns = 'movieId', values = 'rating').fillna(0)
df.head()

movieId,100,100017,100032,100272,100450,101,101362,1018,101904,102,102165,1023,1024,103,103048,103210,103299,103502,103539,103688,103731,104,1040,104211,104241,104374,104863,1049,105,1050,1051,105121,105246,105254,105593,105763,1059,106417,107,107081,...,956,957,95744,95752,95949,95963,96,961,962,963,964,965,966,968,96821,97,97593,976,97936,97938,98,980,981,98122,982,98369,984,98491,986,98604,987,988,99,990,991,99106,992,994,996,99846
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# 환경설정
if torch.cuda.is_available():
  DEVICE = torch.device('cuda')
else:
  DEVICE = torch.device('cpu')
print(DEVICE)

cpu


In [ ]:
BATCH_SIZE = 128
EPOCHS = 10

In [ ]:
R = df.iloc[:, : ].values

user_num = R.shape[0]
item_num = R.shape[1]

In [ ]:
features_idx = []
for i in range(user_num):
  for j in range(item_num):
    features_idx.append([i, j])

In [ ]:
features = np.transpose(R.nonzero()).tolist()

In [ ]:
labels = []

for i, j in features:
  labels.append(R[i, j])

In [ ]:
class GMF_and_MLP(nn.Module):
  def __init__(self, user_num, item_num, factor_num, num_layers = 3):
      super(GMF_and_MLP, self).__init__()

      # GMF 임베딩
      self.GMF_user_embedding = nn.Embedding(user_num, factor_num)
      self.GMF_item_embedding = nn.Embedding(item_num, factor_num)

      # MLP 임베딩
      self.MLP_user_embedding = nn.Embedding(user_num, factor_num)
      self.MLP_item_embedding = nn.Embedding(item_num, factor_num)

      MLP_modules = []
      input_size = factor_num * 2
      for i in range(num_layers):
        MLP_modules.append(nn.Linear(input_size, input_size // 2))
        MLP_modules.append(nn.ReLU())
        input_size //=  2
      self.MLP_layers = nn.Sequential(*MLP_modules)

      # FC
      self.FC_layer = nn.Sequential(
          nn.Linear(factor_num + input_size, 1)
      )
      self._init_weight_()

  def _init_weight_(self):
      # weight 초기화
      nn.init.normal_(self.GMF_user_embedding.weight, std=0.01)
      nn.init.normal_(self.GMF_item_embedding.weight, std=0.01)
      nn.init.normal_(self.MLP_user_embedding.weight, std=0.01)
      nn.init.normal_(self.MLP_item_embedding.weight, std=0.01)

      for m in self.MLP_layers:
          if isinstance(m, nn.Linear):
              nn.init.xavier_uniform_(m.weight)

      for m in self.FC_layer:
          if isinstance(m, nn.Linear):
              nn.init.xavier_uniform_(m.weight)

  def forward(self, user, item):
      GMF_user_embedding = self.GMF_user_embedding(user)
      GMF_item_embedding = self.GMF_item_embedding(item)

      output_GMF = (GMF_user_embedding * GMF_item_embedding)

      MLP_user_embedding = self.MLP_user_embedding(user)
      MLP_item_embedding = self.MLP_item_embedding(item)

      concat_two_latent_vactors = torch.cat((MLP_user_embedding, MLP_item_embedding), -1)
      output_MLP = self.MLP_layers(concat_two_latent_vactors)

      concat_GMF_MLP = torch.cat((output_GMF, output_MLP), -1)

      out = self.FC_layer(concat_GMF_MLP)
      out = out.view(-1)

      return out

In [ ]:
model = GMF_and_MLP(user_num = user_num, item_num = item_num, factor_num = 20).to(DEVICE)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
class NCFData(data.Dataset):
    def __init__(self, features, labels = None):
        super(NCFData, self).__init__()
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):

      if self.labels is not None:
        user = self.features[idx][0]
        item = self.features[idx][1]
        label = self.labels[idx]

        return user, item, label

      else: 
        user = self.features[idx][0]
        item = self.features[idx][1]
        return user, item

In [ ]:
def train(model, train_loader):
  model.train()
  train_loss = 0
  for user, item, label in train_loader:
    user = user.to(DEVICE)
    item = item.to(DEVICE)
    label = label.float().to(DEVICE)
    optimizer.zero_grad()
    output = model(user, item)
    loss = criterion(output, label)
    loss.backward()
    optimizer.step()

    train_loss += loss.item()

  train_loss /= len(train_loader)
  return train_loss

In [ ]:
def predict(model, loader):
  model.eval()
  result = []
  with torch.no_grad():
    for user, item in loader:
      user = user.to(DEVICE)
      item = item.to(DEVICE)
      output = model(user, item)
      result.append(output.view(-1))

  return result

In [ ]:
train_dataset = NCFData(features, labels)
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE)

for epoch in range(1, EPOCHS + 1):
  train_loss = train(model, train_loader)
  print(f"\n[EPOCH: {epoch}], \tTrain Loss: {train_loss:.4f}")


[EPOCH: 1], 	Train Loss: 1.4863

[EPOCH: 2], 	Train Loss: 0.8686

[EPOCH: 3], 	Train Loss: 0.8743

[EPOCH: 4], 	Train Loss: 0.6191

[EPOCH: 5], 	Train Loss: 0.4452

[EPOCH: 6], 	Train Loss: 0.3894

[EPOCH: 7], 	Train Loss: 0.3588

[EPOCH: 8], 	Train Loss: 0.3374

[EPOCH: 9], 	Train Loss: 0.3164

[EPOCH: 10], 	Train Loss: 0.3157


In [ ]:
dataset = NCFData(features_idx)
loader = data.DataLoader(dataset, batch_size = BATCH_SIZE)
pred = predict(model, loader)
pred = torch.cat(pred).view(user_num, item_num)

In [ ]:
pred.shape

torch.Size([671, 2830])

In [ ]:
pred

tensor([[3.3646, 2.9302, 3.2288,  ..., 3.2210, 1.9280, 2.6346],
        [4.0337, 2.3965, 3.0552,  ..., 3.1314, 2.8062, 3.7697],
        [3.6591, 2.5289, 2.9345,  ..., 4.2492, 3.9218, 3.8126],
        ...,
        [1.7532, 2.7788, 3.7487,  ..., 4.1492, 3.3240, 2.7195],
        [3.2449, 3.0934, 3.4631,  ..., 4.0583, 3.6396, 3.5495],
        [3.5456, 2.5031, 3.6707,  ..., 3.5429, 3.6101, 3.9073]])

In [ ]:
R.shape

(671, 2830)

In [ ]:
R

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
_, idx =  torch.topk(pred, k = 10, dim = 1)

In [ ]:
pred[0, 168]

tensor(1.6956)

In [ ]:
R[0, 168]

2.5

데이터의 양이 적고 매우 Sparse한 형태의 데이터이기 때문에 모델의 성능이 매우 좋지 않은 것으로 생각된다.

# GMF and MLP - Watching Data

user one-hot vector와 item one-hot vector는 R에 존재하는 데이터만 활용

input = R에서 존재하는 데이터만 활용 (왜? R에서 존재하는 데이터만 활용하면 결국 user one-hot vector * Latent user vector, item one-hot vector * Latent item vector 을 한것과 동일하게 됨)

GMF Latent user vector = (user_num ,factor)

GMF Latent item vector = (item_num ,factor)

MLP Latent user vector = (user_num ,factor)

MLP Latent item vector = (item_num ,factor)

output_GMF = GMF Latent user vector * GMF Latent item vector

MLP_concat_two_latent_vactors = concat(MLP Latent user vector, MLP Latent item vector)

output_MLP = MLP_layer(MLP_concat_two_latent_vactors)

output = concat(output_GMF, output_MLP)

out = FC_layer(output, 1)

output = Y_ui ~ out

최종 예측 값은 유저가 해당 영화에 줄 평점이 될 것 따라서 이를 바탕으로 영화 추천이 가능

In [1]:
import os
import time
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.backends.cudnn as cudnn

import pandas as pd
import scipy.sparse as sp
import warnings

warnings.filterwarnings(action='ignore')
%matplotlib inline

In [2]:
movie_paths = '/content/drive/MyDrive/추천/data/movies/'

movie = pd.read_csv(movie_paths + "ratings.csv")
meta = pd.read_csv(movie_paths + 'movies_metadata.csv', low_memory=False)
meta = meta.rename(columns={'id':'movieId'})

In [3]:
movie.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
meta.head()

,adult,belongs_to_collection,budget,genres,homepage,movieId,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [5]:
movie['movieId'] = movie['movieId'].astype(str)
meta['movieId'] = meta['movieId'].astype(str)

movie = pd.merge(movie, meta[['movieId', 'original_title']], on='movieId')
movie['watching'] = 1
movie.head()

,userId,movieId,rating,timestamp,original_title,watching
0,1,1371,2.5,1260759135,Rocky III,1
1,4,1371,4.0,949810302,Rocky III,1
2,7,1371,3.0,851869160,Rocky III,1
3,19,1371,4.0,855193404,Rocky III,1
4,21,1371,3.0,853852263,Rocky III,1


In [6]:
df = movie.pivot_table(index='userId', columns = 'movieId', values = 'watching').fillna(0)
df.head()

movieId,100,100017,100032,100272,100450,101,101362,1018,101904,102,102165,1023,1024,103,103048,103210,103299,103502,103539,103688,103731,104,1040,104211,104241,104374,104863,1049,105,1050,1051,105121,105246,105254,105593,105763,1059,106417,107,107081,...,956,957,95744,95752,95949,95963,96,961,962,963,964,965,966,968,96821,97,97593,976,97936,97938,98,980,981,98122,982,98369,984,98491,986,98604,987,988,99,990,991,99106,992,994,996,99846
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# 환경설정
if torch.cuda.is_available():
  DEVICE = torch.device('cuda')
else:
  DEVICE = torch.device('cpu')
print(DEVICE)

cuda


In [8]:
BATCH_SIZE = 128
EPOCHS = 10

In [9]:
R = df.iloc[:, : ].values

user_num = R.shape[0]
item_num = R.shape[1]

In [10]:
features_idx = []
for i in range(user_num):
  for j in range(item_num):
    features_idx.append([i, j])

In [11]:
features = np.transpose(R.nonzero()).tolist()

In [12]:
labels = []

for i, j in features_idx:
  labels.append(R[i, j])

In [13]:
class GMF_and_MLP(nn.Module):
  def __init__(self, user_num, item_num, factor_num, num_layers = 3):
      super(GMF_and_MLP, self).__init__()

      # GMF 임베딩
      self.GMF_user_embedding = nn.Embedding(user_num, factor_num)
      self.GMF_item_embedding = nn.Embedding(item_num, factor_num)

      # MLP 임베딩
      self.MLP_user_embedding = nn.Embedding(user_num, factor_num)
      self.MLP_item_embedding = nn.Embedding(item_num, factor_num)

      MLP_modules = []
      input_size = factor_num * 2
      for i in range(num_layers):
        MLP_modules.append(nn.Linear(input_size, input_size // 2))
        MLP_modules.append(nn.ReLU())
        input_size //=  2
      self.MLP_layers = nn.Sequential(*MLP_modules)

      # FC
      self.FC_layer = nn.Sequential(
          nn.Linear(factor_num + input_size, 1),
          nn.Sigmoid()
      )
      self._init_weight_()

  def _init_weight_(self):
      # weight 초기화
      nn.init.normal_(self.GMF_user_embedding.weight, std=0.01)
      nn.init.normal_(self.GMF_item_embedding.weight, std=0.01)
      nn.init.normal_(self.MLP_user_embedding.weight, std=0.01)
      nn.init.normal_(self.MLP_item_embedding.weight, std=0.01)

      for m in self.MLP_layers:
          if isinstance(m, nn.Linear):
              nn.init.xavier_uniform_(m.weight)

      for m in self.FC_layer:
          if isinstance(m, nn.Linear):
              nn.init.xavier_uniform_(m.weight)

  def forward(self, user, item):
      GMF_user_embedding = self.GMF_user_embedding(user)
      GMF_item_embedding = self.GMF_item_embedding(item)

      output_GMF = (GMF_user_embedding * GMF_item_embedding)

      MLP_user_embedding = self.MLP_user_embedding(user)
      MLP_item_embedding = self.MLP_item_embedding(item)

      concat_two_latent_vactors = torch.cat((MLP_user_embedding, MLP_item_embedding), -1)
      output_MLP = self.MLP_layers(concat_two_latent_vactors)

      concat_GMF_MLP = torch.cat((output_GMF, output_MLP), -1)

      out = self.FC_layer(concat_GMF_MLP)
      out = out.view(-1)

      return out

In [14]:
model = GMF_and_MLP(user_num = user_num, item_num = item_num, factor_num = 20).to(DEVICE)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [15]:
class NCFData(data.Dataset):
    def __init__(self, features, labels = None):
        super(NCFData, self).__init__()
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):

      if self.labels is not None:
        user = self.features[idx][0]
        item = self.features[idx][1]
        label = self.labels[idx]

        return user, item, label

      else: 
        user = self.features[idx][0]
        item = self.features[idx][1]
        return user, item

In [16]:
def train(model, train_loader):
  model.train()
  train_loss = 0
  for user, item, label in train_loader:
    user = user.to(DEVICE)
    item = item.to(DEVICE)
    label = label.float().to(DEVICE)
    optimizer.zero_grad()
    output = model(user, item)
    loss = criterion(output, label)
    loss.backward()
    optimizer.step()

    train_loss += loss.item()

  train_loss /= len(train_loader)
  return train_loss

In [17]:
def predict(model, loader):
  model.eval()
  result = []
  with torch.no_grad():
    for user, item in loader:
      user = user.to(DEVICE)
      item = item.to(DEVICE)
      output = model(user, item)
      result.append(output.view(-1))

  return result

In [18]:
train_dataset = NCFData(features_idx, labels)
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle= True)

for epoch in range(1, EPOCHS + 1):
  train_loss = train(model, train_loader)
  print(f"\n[EPOCH: {epoch}], \tTrain Loss: {train_loss:.4f}")


[EPOCH: 1], 	Train Loss: 0.0743

[EPOCH: 2], 	Train Loss: 0.0636

[EPOCH: 3], 	Train Loss: 0.0603

[EPOCH: 4], 	Train Loss: 0.0589

[EPOCH: 5], 	Train Loss: 0.0583

[EPOCH: 6], 	Train Loss: 0.0579

[EPOCH: 7], 	Train Loss: 0.0578

[EPOCH: 8], 	Train Loss: 0.0575

[EPOCH: 9], 	Train Loss: 0.0576

[EPOCH: 10], 	Train Loss: 0.0577


In [19]:
dataset = NCFData(features_idx)
loader = data.DataLoader(dataset, batch_size = BATCH_SIZE)
pred = predict(model, loader)
pred = torch.cat(pred).view(user_num, item_num)

In [20]:
pred.shape

torch.Size([671, 2830])

In [21]:
pred

tensor([[2.7552e-07, 7.1170e-11, 1.8879e-13,  ..., 1.5908e-06, 1.6034e-07,
         2.9589e-11],
        [6.1797e-07, 5.8171e-12, 1.0347e-13,  ..., 2.2148e-05, 1.8096e-08,
         2.5473e-13],
        [3.6389e-06, 8.1974e-08, 2.6040e-09,  ..., 1.0218e-05, 8.8486e-05,
         5.6360e-09],
        ...,
        [2.0834e-07, 4.3414e-09, 4.9481e-10,  ..., 2.9967e-05, 7.0031e-08,
         3.8819e-10],
        [2.1406e-04, 1.6129e-10, 3.2968e-10,  ..., 4.4024e-04, 2.2731e-06,
         4.9068e-12],
        [5.9553e-07, 1.2189e-09, 4.8112e-10,  ..., 3.4414e-04, 2.0893e-07,
         6.6648e-11]], device='cuda:0')

In [22]:
R.shape

(671, 2830)

In [23]:
R

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
_, idx =  torch.topk(pred, k = 10, dim = 1)

In [25]:
pred[0, 168]

tensor(0.0227, device='cuda:0')

In [26]:
R[0, 168]

1.0

데이터의 양이 적고 매우 Sparse한 형태의 데이터이기 때문에 모델의 성능이 매우 좋지 않은 것으로 생각된다.